In [4]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
    
# Setting this option will print all rows of a dataframe
pd.set_option('display.max_rows', None)

In [5]:
# scraping web page of a particular car

def car_scraping(web_page_link, dictionary):
    
    html_text = requests.get(web_page_link).text
    soup = BeautifulSoup(html_text, 'lxml')

    try:
        brand_model = soup.find('div', class_= 'ab-col-9').text.split(" ")
    except:
        brand_model = np.nan

    try:
        brand = brand_model[0].strip(" \t\n\r")
    except: 
        brand = np.nan

    try:
        model = brand_model[1].strip(" \t\n\r")
    except:
        model = np.nan

    try:
        location = soup.find('div', class_= 'subtitle').find(class_="subtitle__location").text
    except:
        location = np.nan

    dictionary["Značka"] = brand
    dictionary["Model"] = model
    dictionary["Lokalita"] = location

    basic_parameters = soup.find_all('div', class_= 'ab-grid parameters__row')
    for parameter in basic_parameters:
        try:
            parameter_list = parameter.text.split(":")
            key = parameter_list[0].strip(" \t\n\r")
            if key == 'Najazdené':
                value = parameter_list[1].split('km')[0].strip(" \t\n\r")
            else:
                value = parameter_list[1].strip(" \t\n\r")
            dictionary[key] = value
        except:
            dictionary[parameter] = np.nan

    other_parameters = soup.find_all('section', class_= 'tab__section tab__section_underline')
    for par in other_parameters:
        try:
            parameter_list = par.text.strip(" \t\n\r").split("\n")
            for parameter in parameter_list:
                if parameter not in {'','Bezpečnosť', 'Komfort','Dalšia výbava', 'Dalšie informácie'}:
                    if 'Airbag' in parameter:
                        if 'x' in parameter:
                            dictionary['Airbagy'] = parameter
                        else:
                            dictionary['Airbagy'] = "áno"
                    elif 'Rádio' in parameter:
                        dictionary['Autorádio'] = parameter
                    elif 'Elektrické okná' in parameter:
                        dictionary['Elektrické okná'] = parameter
                    elif 'Klimatizácia' in parameter:
                        dictionary["Klimatizácia"] = "áno"
                    elif 'Parkovacie senzory' in parameter:
                        dictionary["Parkovacie senzory"]  = parameter
                    elif 'Vyhrievané sedačky' in parameter:
                        dictionary['Vyhrievané sedačky']  = parameter
                    elif 'STK platná' in parameter:
                        pass
                    elif 'EK platná' in parameter:
                        pass    
                    else:
                        dictionary[parameter] = "áno"
        except:
            dictionary[par] = np.nan
    try:
        price = soup.find('h2', class_= 'p-amount').text
    except:
        price = np.nan
    dictionary["Cena"] = price

In [21]:
# final list containing all scraped cars, each car is represented by a dictionary 
final_cars_list = []

for i in range(1,783):
    
    page = f"https://osobne-auta.autobazar.sk/?p[order]=23&p[page]={i}"
    html_text = requests.get(page).text
    soup = BeautifulSoup(html_text, 'lxml')

    # scraping cars on a single page
    cars = soup.find_all('div', class_= 'item-teaser text')
    for car in cars:
        car_parameters = {}
        
        heading = car.find('h2')

        if heading == "":
            url = np.nan
            brand = np.nan
            model = np.nan
        else:
            url = heading.find('a', href=True)['href']
            brand = heading.text.split(" ")[0]
            model = heading.text.split(" ")[1]

        try:
            location = car.find_all('span', class_="half")[-1].text.strip(" \t\n\r")
        except:
            location = np.nan

        car_parameters["Názov"] = heading.text
        car_parameters["Značka"] = brand
        car_parameters["Model"] = model
        car_parameters["Kraj"] = location
        car_parameters["URL"] = url
        
        car_scraping(url, car_parameters)
        
        final_cars_list.append(car_parameters)

    print(f"Page {i} has been scraped successfully.")

Page 1 has been scraped successfully.
Page 2 has been scraped successfully.
Page 3 has been scraped successfully.
Page 4 has been scraped successfully.
Page 5 has been scraped successfully.
Page 6 has been scraped successfully.
Page 7 has been scraped successfully.
Page 8 has been scraped successfully.
Page 9 has been scraped successfully.
Page 10 has been scraped successfully.
Page 11 has been scraped successfully.
Page 12 has been scraped successfully.
Page 13 has been scraped successfully.
Page 14 has been scraped successfully.
Page 15 has been scraped successfully.
Page 16 has been scraped successfully.
Page 17 has been scraped successfully.
Page 18 has been scraped successfully.
Page 19 has been scraped successfully.
Page 20 has been scraped successfully.
Page 21 has been scraped successfully.
Page 22 has been scraped successfully.
Page 23 has been scraped successfully.
Page 24 has been scraped successfully.
Page 25 has been scraped successfully.
Page 26 has been scraped successfu

In [22]:
# transforming (final) list of dictionaries to dataframe
data_cars = pd.DataFrame(data=final_cars_list)
data_cars.head(3)

,Názov,Značka,Model,Kraj,URL,Lokalita,Rok výroby,Stav,Najazdené,Palivo,Objem,Výkon,Prevodovka,Karoséria,Pohon,Norma,Dvere,Miest,Farba,VIN,ABS,Alarm,Centrálne zamykanie,DSC,ESP,Imobilizér,Isofix,Adaptívny tempomat,Parkovacia kamera,Deaktivácia airbagov,Systém kontroly tlaku v pneumatikách (TPMS),Natáčacie svetlomety,LED svetlomety,Diaľkové ovládanie zamykania,Elektrické sedadlá,Elektrické zrkadlá,Kožený interiér,Lakťová opierka,Multifunkčný volant,Navigačný systém,Nezávislé kúrenie,Palubný počítač,Posilňovač riadenia,Tónované sklá,Vyhrievané predné sklo,Vyhrievané zrkadlá,Bezkľúčové štartovanie,Bezkľúčové otváranie dverí,Bluetooth handsfree,LED svietenie,Parkovacie senzory,Vyhrievaný volant,Odvetrávané sedadlá,Elektrické ovládanie kufra,Vzduchové odpruženie podvozku,Pamäťové sedačky,Welcome light,Klimatizácia,Elektrické okná,Autorádio,Xenónové reflektory,Vyhrievané sedačky,Dažďový senzor,Hliníkové disky,Hmlovky,Ťažné zariadenie,Stop&start systém,Panoramatická strecha,Ostrekovače svetlometov,Doťahovanie dverí,Head-up display,Dovezené,Možná výmena,Cena,ASR,Brzdový asistent,EBD,EBV,MSR,Fólie,Tempomat,Sezónne prezutie,Strešný nosič,Svetelný senzor,Garážované,Top stav!,ADS,Airbagy,Parkovací asistent,1.majiteľ,Ako nové kúpené v SR,Servisná knižka,Úplná servisná história,Klimatizovaná priehradka,Sedadlá s masážnou funkciou,Apple CarPlay,Android Auto,Strešné okno,"TV,LCD panel",Nočné videnie,Kontrola originality,Nehavarované,Vozidlo je v záruke,Deliaca mreža,CD menič,Bezdotykové otváranie kufra,Bi-Xenónové svetlomety,Dotykový displej,STK,EK,Kožený paket,Asistent jazdných pruhov,Kontrola mŕtveho uhla,Asistent rozjazdu do kopca,Mechanické zabezpečenie,Adaptívny podvozok,Xenónové svetlomety,ODO-Pass,Tuning,EČV,Bez EČ,Pravostranné riadenie,Defektný motor alebo prevodovka,Úžitková verzia N1,Na dovoz,Veterán,865342
0,BMW X5 M50d A/T,BMW,X5,ZA kraj,https://www.autobazar.sk/25976565/bmw-x5-m50d-...,Žilina,2015,Používané,265 000,Diesel,2993 cm³ (3 l),280.00 kw (381 PS),8-st. automatická,Combi,4x4,Euro 6,5-dverové,5-miestne,Biela,WBAKS810700M50953,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,Parkovacie senzory,áno,áno,áno,áno,áno,áno,áno,Elektrické okná 4x,Rádio,áno,Vyhrievané sedačky vpredu a vzadu,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,32 900 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Škoda Octavia Combi 1.6 TDI CR DPF Ambition,Škoda,Octavia,TT kraj,https://www.autobazar.sk/25976566/skoda-octavi...,Dunajská Streda,2012,Používané,189 300,Diesel,1598 cm³ (1.6 l),77.00 kw (105 PS),5-st. manuálna,Combi,Predný,Euro 5,5-dverové,5-miestne,Šedá/ sivá,TMBHT21Z8C2096813,áno,NaN,áno,áno,áno,áno,áno,NaN,NaN,áno,áno,NaN,NaN,NaN,NaN,áno,NaN,NaN,NaN,NaN,NaN,áno,áno,áno,NaN,áno,NaN,NaN,áno,NaN,NaN,NaN,NaN,NaN,áno,NaN,NaN,áno,Elektrické okná 2x,Rádio + CD + MP3,NaN,NaN,áno,áno,áno,NaN,áno,NaN,NaN,NaN,NaN,NaN,NaN,6 900 €,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mitsubishi Outlander 2.0 MIVEC Intense+ 4WD CVT,Mitsubishi,Outlander,ZA kraj,https://www.autobazar.sk/25976561/mitsubishi-o...,Martin,12/2016,Používané,50 600,Benzín,1998 cm³ (2 l),110.00 kw (150 PS),Automatická bezstupňová,Combi,4x4,Euro 6,5-dverové,7-miestne,Bieložltá metalíza,JMBXTGF7WHZ000260,áno,NaN,áno,áno,áno,áno,áno,NaN,áno,áno,áno,NaN,áno,áno,NaN,áno,NaN,áno,áno,NaN,NaN,áno,áno,áno,NaN,áno,áno,áno,áno,áno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,áno,Elektrické okná 4x,Rádio + CD + MP3,NaN,Vyhrievané sedačky vpredu,áno,áno,áno,áno,áno,NaN,áno,NaN,NaN,NaN,NaN,18 900 €,NaN,áno,áno,áno,NaN,áno,NaN,áno,áno,NaN,NaN,NaN,áno,Airbag 8x,áno,áno,áno,áno,áno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [24]:
# reorder columns
columns = ['VIN', 'Názov', 'Značka', 'Model', 'URL', 'Kraj', 'Lokalita', 'Rok výroby', 'Stav', 'Najazdené', 'Palivo', 'Objem', 'Výkon', 'Prevodovka', 'Karoséria', 'Pohon', 'Dvere', 'Miest', 'Farba', 
                 'ABS', 'Airbagy', 'Adaptívny tempomat', 'ADS', 'Alarm', 'ASR', 'Brzdový asistent', 'Centrálne zamykanie', 'Deaktivácia airbagov', 'DSC', 'EBD', 'EBV', 'ESP', 'Imobilizér', 'Isofix', 'Mechanické zabezpečenie', 'MSR',
                 'Natáčacie svetlomety', 'Parkovacia kamera', 'Systém kontroly tlaku v pneumatikách (TPMS)', 'Apple CarPlay', 'Android Auto', 'Welcome light', 'Autorádio', 'Adaptívny podvozok', 'Bezdotykové otváranie kufra', 'Bezkľúčové otváranie dverí', 
                 'Bezkľúčové štartovanie', 'Bluetooth handsfree', 'CD menič', 'Diaľkové ovládanie zamykania', 'Dotykový displej', 'Elektrické okná', 'Elektrické sedadlá', 'Elektrické zrkadlá', 'Fólie', 'Klimatizácia', 
                 'Klimatizovaná priehradka', 'Kožený interiér', 'Lakťová opierka', 'LED svetlomety', 'LED svietenie', 'Multifunkčný volant', 'Navigačný systém', 'Palubný počítač', 'Pamäťové sedačky', 'Parkovacie senzory', 'Parkovací asistent', 
                 'Posilňovač riadenia', 'Sedadlá s masážnou funkciou', 'Tempomat', 'Tónované sklá', 'Vyhrievané predné sklo', 'Vyhrievané sedačky', 'Vyhrievané zrkadlá', 'Vyhrievaný volant', 'Vzduchové odpruženie podvozku',
                  'Asistent jazdných pruhov', 'Asistent rozjazdu do kopca', 'Kontrola mŕtveho uhla', 'Nočné videnie', 'Ostrekovače svetlometov', 'Stop&start systém', 'Strešné okno', 'Strešný nosič', 'Ťažné zariadenie', '1.majiteľ', 'Garážované', 'Nehavarované', 'ODO-Pass', 'Servisná knižka', 'Úplná servisná história',
                  'Vozidlo je v záruke', 'Cena']


In [25]:
# reorder columns
data_cars = data_cars[columns]
data_cars.head()

,VIN,Názov,Značka,Model,URL,Kraj,Lokalita,Rok výroby,Stav,Najazdené,Palivo,Objem,Výkon,Prevodovka,Karoséria,Pohon,Dvere,Miest,Farba,ABS,Airbagy,Adaptívny tempomat,ADS,Alarm,ASR,Brzdový asistent,Centrálne zamykanie,Deaktivácia airbagov,DSC,EBD,EBV,ESP,Imobilizér,Isofix,Mechanické zabezpečenie,MSR,Natáčacie svetlomety,Parkovacia kamera,Systém kontroly tlaku v pneumatikách (TPMS),Apple CarPlay,Android Auto,Welcome light,Autorádio,Adaptívny podvozok,Bezdotykové otváranie kufra,Bezkľúčové otváranie dverí,Bezkľúčové štartovanie,Bluetooth handsfree,CD menič,Diaľkové ovládanie zamykania,Dotykový displej,Elektrické okná,Elektrické sedadlá,Elektrické zrkadlá,Fólie,Klimatizácia,Klimatizovaná priehradka,Kožený interiér,Lakťová opierka,LED svetlomety,LED svietenie,Multifunkčný volant,Navigačný systém,Palubný počítač,Pamäťové sedačky,Parkovacie senzory,Parkovací asistent,Posilňovač riadenia,Sedadlá s masážnou funkciou,Tempomat,Tónované sklá,Vyhrievané predné sklo,Vyhrievané sedačky,Vyhrievané zrkadlá,Vyhrievaný volant,Vzduchové odpruženie podvozku,Asistent jazdných pruhov,Asistent rozjazdu do kopca,Kontrola mŕtveho uhla,Nočné videnie,Ostrekovače svetlometov,Stop&start systém,Strešné okno,Strešný nosič,Ťažné zariadenie,1.majiteľ,Garážované,Nehavarované,ODO-Pass,Servisná knižka,Úplná servisná história,Vozidlo je v záruke,Cena
0,WBAKS810700M50953,BMW X5 M50d A/T,BMW,X5,https://www.autobazar.sk/25976565/bmw-x5-m50d-...,ZA kraj,Žilina,2015,Používané,265 000,Diesel,2993 cm³ (3 l),280.00 kw (381 PS),8-st. automatická,Combi,4x4,5-dverové,5-miestne,Biela,áno,NaN,áno,NaN,áno,NaN,NaN,áno,áno,áno,NaN,NaN,áno,áno,áno,NaN,NaN,áno,áno,áno,NaN,NaN,áno,Rádio,NaN,NaN,áno,áno,áno,NaN,áno,NaN,Elektrické okná 4x,áno,áno,NaN,áno,NaN,áno,áno,áno,áno,áno,áno,áno,áno,Parkovacie senzory,NaN,áno,NaN,NaN,áno,áno,Vyhrievané sedačky vpredu a vzadu,áno,áno,áno,NaN,NaN,NaN,NaN,áno,áno,NaN,NaN,áno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32 900 €
1,TMBHT21Z8C2096813,Škoda Octavia Combi 1.6 TDI CR DPF Ambition,Škoda,Octavia,https://www.autobazar.sk/25976566/skoda-octavi...,TT kraj,Dunajská Streda,2012,Používané,189 300,Diesel,1598 cm³ (1.6 l),77.00 kw (105 PS),5-st. manuálna,Combi,Predný,5-dverové,5-miestne,Šedá/ sivá,áno,NaN,NaN,NaN,NaN,áno,áno,áno,áno,áno,áno,áno,áno,áno,áno,NaN,áno,NaN,NaN,áno,NaN,NaN,NaN,Rádio + CD + MP3,NaN,NaN,NaN,NaN,áno,NaN,NaN,NaN,Elektrické okná 2x,NaN,áno,áno,áno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,áno,NaN,NaN,NaN,áno,NaN,áno,áno,NaN,NaN,áno,NaN,áno,NaN,NaN,NaN,NaN,NaN,áno,NaN,áno,NaN,NaN,áno,NaN,NaN,NaN,NaN,NaN,6 900 €
2,JMBXTGF7WHZ000260,Mitsubishi Outlander 2.0 MIVEC Intense+ 4WD CVT,Mitsubishi,Outlander,https://www.autobazar.sk/25976561/mitsubishi-o...,ZA kraj,Martin,12/2016,Používané,50 600,Benzín,1998 cm³ (2 l),110.00 kw (150 PS),Automatická bezstupňová,Combi,4x4,5-dverové,7-miestne,Bieložltá metalíza,áno,Airbag 8x,NaN,áno,NaN,NaN,áno,áno,áno,áno,áno,áno,áno,áno,áno,NaN,NaN,NaN,áno,áno,NaN,NaN,NaN,Rádio + CD + MP3,NaN,NaN,áno,áno,áno,NaN,áno,NaN,Elektrické okná 4x,NaN,áno,áno,áno,NaN,NaN,áno,áno,áno,áno,NaN,áno,NaN,NaN,áno,áno,NaN,NaN,áno,NaN,Vyhrievané sedačky vpredu,áno,NaN,NaN,NaN,NaN,NaN,NaN,áno,áno,NaN,áno,áno,áno,NaN,NaN,NaN,áno,áno,NaN,18 900 €
3,TMBLN9NS1R8016736,Škoda Kodiaq 2.0 TDI SCR EVO 200 L&K 4x4 DSG,Škoda,Kodiaq,https://www.autobazar.sk/25976452/skoda-kodiaq...,PO kraj,Bardejov,5/2023,Nové,307,Diesel,1968 cm³ (2 l),147.00 kw (200 PS),7-st. automatická,Combi,4x4,5-dverové,5-miestne,Šedá/ sivá metalíza,NaN,Airbag 10x,áno,áno,NaN,NaN,NaN,NaN,NaN,NaN,áno,áno,NaN,NaN,NaN,NaN,NaN,áno,áno,NaN,áno,áno,áno,Rádio,NaN,NaN,áno,NaN,NaN,NaN,áno,NaN,Elektrické okná 4x,NaN,NaN,áno,NaN,áno,NaN,áno,NaN,áno,áno,NaN,áno,áno,Parkovacie senzory,áno,áno,áno,NaN,NaN,áno,Vyhrievané sedačky vpredu a vzadu,NaN,áno,NaN,NaN,NaN,NaN,áno,NaN,áno,áno,áno,áno,áno,áno,áno,NaN,áno,áno,áno,57 990 €
4,WVGZZZ1TZKW018751,Volkswagen Touran 2.0 TDI SCR 150k Highline,Volkswagen,Touran,https://www.autobazar.sk/25976539/volkswagen-t...,TN kraj,Púchov,2019,Používané,191 100,Diesel,1968 cm³

In [26]:
# save to csv
data_cars.to_csv("cars.csv")